In [1]:
# Import required libraries for data manipulation and analysis
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import time
from scipy.cluster import hierarchy
from scipy.spatial.distance import squareform
from scipy.stats import spearmanr

In [2]:
#Import required sklearn functions
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import VarianceThreshold
from collections import defaultdict

In [3]:
#Import sklearn classifiers
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression

In [4]:
#Import library to oversample 
from imblearn.over_sampling import RandomOverSampler

In [5]:
#Import RDKit and Mordred libraries
from rdkit import Chem
from rdkit.Chem import Draw
from mordred import Calculator, descriptors

In [6]:
#Allows figures to be visualized in jupyter notebook
%matplotlib inline

In [7]:
#Functions used in the study

#Remove those numbers from analysis data
def filter_rows_by_values1(df, col, values):
    return df[~df[col].isin(values)]

#Remove those numbers from analysis data
def filter_rows_by_values2(df, col, values):
    return df[df[col].isin(values)]

#Get Mordred calcs
def get_Mordred(data_input):
    # Assigns Reactants Mordred Info
    reactants = data_input['Substrate']
    
    reactants_mol_list = []
    for inChi_reactants in reactants:
      reactants_mol = Chem.MolFromInchi(inChi_reactants)
      reactants_mol_list.append(reactants_mol)

    # Puts reactants into Pandas Type
    reactant_data = []
    reactant_data = calc.pandas(reactants_mol_list)
       
    #Joins Mordred parameters with experimental, atomic charges, and JChem for Excel parameters
    add_reactants = pd.concat((data_input, reactant_data), axis=1)
    
    #Force any non-numeric entries as NaN and replace them with 0
    int_data = add_reactants.apply(pd.to_numeric, errors='coerce')
    
    output = int_data.fillna(0)#, inplace=True)

    return output

#Remove zero varience
def remove_zero_varience(values):
   sel = VarianceThreshold()
   _ = sel.fit(values)
   mask = sel.get_support()
   values = values.loc[:,mask] 
   return values

def remove_95correlated(correlated):
    #Remove any features that are greater than 95% correlated
    corr_matrix = correlated.corr()
    upper_tri = corr_matrix.where(np.triu(np.ones(corr_matrix.shape),k=1).astype(np.bool))

    to_drop = [column for column in upper_tri.columns if any(upper_tri[column] > 0.95)]

    correlated = correlated.drop(to_drop, axis = 1)
    corr_matrix = correlated.corr()
    return correlated

def remove_nonimportant(X_values, y_values):
    # Specifys Random Forest and the Number of Trees, SelectFromModel will
    # select features which are most important
    feature_names = [f"feature {i}" for i in range(X_values.shape[1])]
    forest = RandomForestClassifier(random_state=42)
    forest.fit(X_values, y_values)

    start_time = time.time()
    importances = forest.feature_importances_
    std = np.std([tree.feature_importances_ for tree in forest.estimators_], axis=0)
    elapsed_time = time.time() - start_time

    threshold = np.sort(importances)[-100]
    
    sel = SelectFromModel(RandomForestClassifier(n_estimators = 800, max_depth=30),threshold=threshold)
    sel.fit(X_values, y_values)

    # Select the final features set 
    sel.get_support()
    selected_feat= X_values.columns[(sel.get_support())]

    # Prints the names of the final selected features
    print(selected_feat)
    X_values = X_values[selected_feat]
    
    return X_values

def dendrogram(X_values, y):
    corr = spearmanr(X_values).correlation
    # Ensure the correlation matrix is symmetric
    corr = (corr + corr.T) / 2
    np.fill_diagonal(corr, 1)
    distance_matrix = 1 - np.abs(corr)
    dist_linkage = hierarchy.ward(squareform(distance_matrix))
  
    trained_cluster_ids = hierarchy.fcluster(dist_linkage, y, criterion="distance")
    trained_cluster_id_to_feature_ids = defaultdict(list) 
    for idx, trained_cluster_id in enumerate(trained_cluster_ids):
        trained_cluster_id_to_feature_ids[trained_cluster_id].append(idx)
    
    trained_selected_features = [v[0] for v in trained_cluster_id_to_feature_ids.values()]
    final_selected_features = X_values.columns[trained_selected_features]
    X_train = X_values[final_selected_features]
    return X_train

def classificationMetrics(results, y_test, pred):
    acc = accuracy_score(y_test, pred)
    prec = precision_score(y_test, pred, average=None, zero_division=0)
    recall = recall_score(y_test, pred, average=None)
    F1 = f1_score(y_test, pred, average=None)
    comb = np.concatenate((x, y, acc, prec, recall, F1), axis=None)
    comb = [comb]
    results = results.append(pd.DataFrame(comb, columns=results.columns), ignore_index=True)
    return results

def averageMetrics(df, col):
    acc_mean = df.groupby(col)['acc'].mean()
    precision_0_mean = df.groupby(col)['precision 0'].mean()
    precision_1_mean = df.groupby(col)['precision 1'].mean()
    recall_0_mean = df.groupby(col)['recall 0'].mean()
    recall_1_mean = df.groupby(col)['recall 1'].mean()
    F1_0_mean = df.groupby(col)['F1 0'].mean()
    F1_1_mean = df.groupby(col)['F1 1'].mean()

    ave_df = pd.concat([acc_mean , precision_0_mean, precision_1_mean, recall_0_mean, 
                        recall_1_mean, F1_0_mean, F1_1_mean], axis=1)
    return ave_df
    
    

In [8]:
# Sets Pandas Display to Monitor Code
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', 200)

In [9]:
# Create Mordred Calculator
calc = Calculator(descriptors, ignore_3D=True)

In [10]:
# Read Training/Test data input File
data = pd.read_csv('BorylationTrainingTest 9-12-24.csv')
data.head()

,Substrate,Product,Boronic Ester,Active Catalyst-Ligand,Catalyst,Ligand,Buried_Vol,PyramidalizationAR,PyramidalizationG,SASA_area,SASA_vol,Sterimol_L,SterimolB_1,Sterimol_B_5,Buried_Sterimol_L,Buried_SterimolB_1,Buried_Sterimol_B_5,Hirshfeld Heavy Atom Charge,CM5 Charge,Hirshfeld Carbon Charge,Hirshfeld Hydrogen Charge,ESP Heavy Atom Charge,ESP Carbon Charge,ESP Hydrogen Charge,NPA Carbon Charge,NPA Hydrogen Charge,MBS Heavy Atom Charge,MBS Carbon Charge,MBS Hydrogen Charge,Mulliken Heavy Charge,Mulliken Carbon Charge,Mulliken Hydrogen Charge,Solvent,Temp,Aliphatic Atom Count,Aliphatic Bond Count,Aliphatic Ring Count,Aromatic Atom Count,Aromatic Bond Count,Aromatic Ring Count,Steric Effect Index,Atomic_Polarizability,Balaban Index,Chain Atom Count,Distance Degree,Dreiding Energy,Eccentricity,Harary Index,Hydrogen Acceptor Count,Hydrogen Acceptor Site Count,Heteroatom Aromatic Ring Count,Hydrogen Donor Count,Hydrogen Donor Site Count,Hyper Wiener Index,Largest Ring Size,Max Projection Area,Max Projection Radius,MaxZ,Moleculare Polarizability,Platt Index,Refractivity,Ring Atom Count,Rot Bond Count,Sigma Electronegativity,Wiener Index,Product_Ratio
0,"InChI=1S/C8H18/c1-3-5-7-8-6-4-2/h3-8H2,1-2H3",InChI=1S/C14H29BO2/c1-6-7-8-9-10-11-12-15-16-1...,2,6,1,1,43.1,0.683535,10.223844,735.602381,1441.376195,5.941608,4.79083,7.558608,5.941608,4.79083,7.2781,-0.003389,-0.000578,-0.101759,0.032223,-0.078389,-0.182602,0.032537,-0.57256,0.19272,-0.008908,-0.254190,0.080765,-0.220320,-0.621212,0.132089,2,150,8,7,0,0,0,0,1.193207,1.116291,2.53006,8,28,12.13,7,13.742857,0,0,0,0,0,210,0,52.554143,6.587896,5.713901,15.52291,12,38.6102,0,5,7.387931,84,1
1,"InChI=1S/C8H18/c1-3-5-7-8-6-4-2/h3-8H2,1-2H3",InChI=1S/C14H29BO2/c1-7-8-9-10-11-12(2)15-16-1...,2,6,1,1,43.1,0.683535,10.223844,735.602381,1441.376195,5.941608,4.79083,7.558608,5.941608,4.79083,7.2781,0.003094,0.000249,-0.057714,0.030404,0.098246,0.182934,-0.042344,-0.38255,0.18757,0.009321,-0.138428,0.073874,0.044556,-0.224805,0.134680,2,150,8,7,0,0,0,0,2.190292,1.116291,2.53006,8,22,12.13,6,13.742857,0,0,0,0,0,210,0,52.554143,6.587896,5.713901,15.52291,12,38.6102,0,5,7.470532,84,0
2,"InChI=1S/C8H18/c1-3-5-7-8-6-4-2/h3-8H2,1-2H3",InChI=1S/C14H29BO2/c1-7-9-10-11-12(8-2)15-16-1...,2,6,1,1,43.1,0.683535,10.223844,735.602381,1441.376195,5.941608,4.79083,7.558608,5.941608,4.79083,7.2781,-0.000052,-0.000032,-0.059257,0.029602,-0.011526,0.182934,-0.012016,-0.37899,0.18768,-0.000222,-0.141245,0.070512,0.012779,-0.261495,0.137137,2,150,8,7,0,0,0,0,2.310662,1.116291,2.53006,8,18,12.13,5,13.742857,0,0,0,0,0,210,0,52.554143,6.587896,5.713901,15.52291,12,38.6102,0,5,7.493894,84,0
3,"InChI=1S/C8H18/c1-3-5-7-8-6-4-2/h3-8H2,1-2H3",InChI=1S/C14H29BO2/c1-7-9-11-12(10-8-2)15-16-1...,2,6,1,1,43.1,0.683535,10.223844,735.602381,1441.376195,5.941608,4.79083,7.558608,5.941608,4.79083,7.2781,0.000299,0.000312,-0.058954,0.029626,-0.008331,0.037009,-0.022670,-0.37717,0.18811,-0.000191,-0.141973,0.070891,0.162986,-0.115385,0.139186,2,150,8,7,0,0,0,0,2.339699,1.116291,2.53006,8,16,12.13,4,13.742857,0,0,0,0,0,210,0,52.554143,6.587896,5.713901,15.52291,12,38.6102,0,5,7.496149,84,0
4,InChI=1S/C12H27N/c1-4-7-10-13(11-8-5-2)12-9-6-...,InChI=1S/C18H38BNO2/c1-7-9-14-20(15-10-8-2)16-...,2,6,2,1,43.1,0.683535,10.223844,735.602381,1441.376195,5.941608,4.79083,7.558608,5.941608,4.79083,7.2781,-0.001043,0.001811,-0.100893,0.032591,-0.063472,-0.138162,0.018852,-0.57090,0.19285,-0.006285,-0.253652,0.081094,-0.198326,-0.602542,0.133170,3,150,13,12,0,0,0,0,1.210289,1.116291,3.46799,13,62,34.13,8,30.132143,1,1,0,0,0,885,0,80.471539,6.502117,6.274216,24.42432,24,61.6089,0,9,7.388345,300,1


In [11]:
#group the compounds by numbers
data['grouped'] = data.groupby('Substrate', sort=False).ngroup()
data[['Substrate','grouped']]

,Substrate,grouped
0,"InChI=1S/C8H18/c1-3-5-7-8-6-4-2/h3-8H2,1-2H3",0
1,"InChI=1S/C8H18/c1-3-5-7-8-6-4-2/h3-8H2,1-2H3",0
2,"InChI=1S/C8H18/c1-3-5-7-8-6-4-2/h3-8H2,1-2H3",0
3,"InChI=1S/C8H18/c1-3-5-7-8-6-4-2/h3-8H2,1-2H3",0
4,InChI=1S/C12H27N/c1-4-7-10-13(11-8-5-2)12-9-6-...,1
...,...,...
1028,InChI=1S/C15H18/c1-10(2)13-7-5-11(3)14-8-6-12(...,200
1029,InChI=1S/C15H18/c1-10(2)13-7-5-11(3)14-8-6-12(...,200
1030,InChI=1S/C15H18/c1-10(2)13-7-5-11(3)14-8-6-12(...,200
1031,InChI=1S/C15H18/c1-10(2)13-7-5-11(3)14-8-6-12(...,200


In [12]:
#Convert substrates to Mordred features
data = get_Mordred(data)

100%|██████████| 1033/1033 [01:40<00:00, 10.26it/s]


In [13]:
MLPResults_df = pd.DataFrame(columns = ['x', 'y', 'acc', 'precision 0',
                                   'precision 1','recall 0', 'recall 1', 'F1 0', 'F1 1'])

SVM1Results_df = pd.DataFrame(columns =  ['x', 'y', 'acc', 'precision 0',
                                   'precision 1','recall 0', 'recall 1', 'F1 0', 'F1 1'])

SVM2Results_df = pd.DataFrame(columns =  ['x', 'y', 'acc', 'precision 0',
                                   'precision 1','recall 0', 'recall 1', 'F1 0', 'F1 1'])

GaussResults_df = pd.DataFrame(columns =  ['x', 'y', 'acc', 'precision 0',
                                   'precision 1','recall 0', 'recall 1', 'F1 0', 'F1 1'])

DTResults_df = pd.DataFrame(columns =  ['x', 'y', 'acc', 'precision 0',
                                   'precision 1','recall 0', 'recall 1', 'F1 0', 'F1 1'])

adaResults_df = pd.DataFrame(columns =  ['x', 'y', 'acc', 'precision 0',
                                   'precision 1','recall 0', 'recall 1', 'F1 0', 'F1 1'])

NBResults_df = pd.DataFrame(columns =  ['x', 'y', 'acc', 'precision 0',
                                   'precision 1','recall 0', 'recall 1', 'F1 0', 'F1 1'])

LRResults_df = pd.DataFrame(columns =  ['x', 'y', 'acc', 'precision 0',
                                   'precision 1','recall 0', 'recall 1', 'F1 0', 'F1 1'])

MLPmaxacc_comb = pd.DataFrame()
SVM1maxacc_comb = pd.DataFrame()
SVM2maxacc_comb = pd.DataFrame()
Gaussmaxacc_comb = pd.DataFrame()
DTmaxacc_comb = pd.DataFrame()
adamaxacc_comb = pd.DataFrame()
NBmaxacc_comb = pd.DataFrame()
LRmaxacc_comb = pd.DataFrame()

model_columns = pd.DataFrame()
for_range = range(1, 11)
for x in for_range:
    #Get numbers to represent compounds
    arr = np.arange(0, 200,  dtype=int)

    #Get 20% of numbers, without replacement
    set_numbers = np.random.choice(arr, int(len(arr)*0.20), replace=False ) 
    
    #Seperate training (80%) and test data (20%)
    training_data = filter_rows_by_values1(data, "grouped", set_numbers)
    training_data = training_data.drop('grouped', axis = 1)    
    test_data = filter_rows_by_values2(data, "grouped", set_numbers)
    test_data = test_data.drop('grouped', axis = 1)     
     
    #Remove features that dont change
    training_data = remove_zero_varience(training_data)
    
    #Remove features that are more than 95% correlated
    training_data = remove_95correlated(training_data)
    
    # Seperate dataset as response variable (Product Ratio) and feature variables
    #Note: Product Ratio is described as "0" for non-borylating sites and "1" for borylating sites
    training_X = training_data.drop('Product_Ratio' , axis = 1)
    training_y = training_data['Product_Ratio']
    test_X = test_data.drop('Product_Ratio' , axis = 1)
    test_y = test_data['Product_Ratio']
    
    #Remove features that are considered less important
    feature_names = [f"feature {i}" for i in range(training_X.shape[1])]
    forest = RandomForestClassifier(random_state=42)
    forest.fit(training_X, training_y)
    
    start_time = time.time()
    importances = forest.feature_importances_
    std = np.std([tree.feature_importances_ for tree in forest.estimators_], axis=0)
    elapsed_time = time.time() - start_time
    
    threshold = np.sort(importances)[-100] 
    sel = SelectFromModel(RandomForestClassifier(n_estimators = 800, max_depth=30),threshold=threshold)
    sel.fit(training_X, training_y)
     
    # Select the reduced features set 
    sel.get_support()
    selected_feat= training_X.columns[(sel.get_support())]
    
    reduced1_X = training_X[selected_feat]
    test_X = test_X[selected_feat]
    
    #Apply over-sampling to dataset
    ros = RandomOverSampler(random_state=10)
    X_resampled, y_resampled = ros.fit_resample(reduced1_X, training_y) 
    
    for y in [0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25,
              0.30, 0.30, 0.30, 0.30, 0.30, 0.30, 0.30, 0.30, 0.30, 0.30, 
              0.35, 0.35, 0.35, 0.35, 0.35, 0.35, 0.35, 0.35, 0.35, 0.35,
              0.40, 0.40, 0.40, 0.40, 0.40, 0.40, 0.40, 0.40, 0.40, 0.40, 
              0.45, 0.45, 0.45, 0.45, 0.45, 0.45, 0.45, 0.45, 0.45, 0.45,
              0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50,
              0.55, 0.55, 0.55, 0.55, 0.55, 0.55, 0.55, 0.55, 0.55, 0.55
             ]:
    
        #Make final training and test set and save them as df's  
        X_train = dendrogram(X_resampled, y)
        test_X = test_X[X_train.columns]
        training_columns_list = X_train.columns.tolist()
        training_columns_list = (x, y, training_columns_list)
        training_columns_list = (pd.DataFrame(training_columns_list).T)

        #MLP CLassifier
        MLP = MLPClassifier(alpha=1, max_iter=1000)
        MLP.fit(X_train, y_resampled)
        pred_MLP = MLP.predict(test_X)
        MLPResults_df = classificationMetrics(MLPResults_df, test_y, pred_MLP)
        
        #Determine the mean accuracy of the different dendrogram settings
        acc_mean = MLPResults_df.groupby('y')['acc'].mean()
        acc_std = MLPResults_df.groupby('y')['acc'].std()
        precision_0_mean = MLPResults_df.groupby('y')['precision 0'].mean()
        precision_0_std = MLPResults_df.groupby('y')['precision 0'].std()        
        precision_1_mean = MLPResults_df.groupby('y')['precision 1'].mean()
        precision_1_std = MLPResults_df.groupby('y')['precision 1'].std()
        recall_0_mean = MLPResults_df.groupby('y')['recall 0'].mean()
        recall_0_std = MLPResults_df.groupby('y')['recall 0'].std()      
        recall_1_mean = MLPResults_df.groupby('y')['recall 1'].mean()
        recall_1_std = MLPResults_df.groupby('y')['recall 1'].std()
        F1_0_mean = MLPResults_df.groupby('y')['F1 0'].mean()
        F1_0_std = MLPResults_df.groupby('y')['F1 0'].std()
        F1_1_mean = MLPResults_df.groupby('y')['F1 1'].mean()
        F1_1_std = MLPResults_df.groupby('y')['F1 1'].std()
        
        MLPaverage_df = pd.concat([acc_mean , acc_std, 
                                   precision_0_mean, precision_0_std, 
                                   precision_1_mean, precision_1_std, 
                                   recall_0_mean, recall_0_std, 
                                   recall_1_mean, recall_1_std,
                                   F1_0_mean, F1_0_std,
                                   F1_1_mean, F1_1_std], axis=1)

        MLPaverage_df.columns = ['acc_mean' , 'acc_std', 'precision_0_mean', 'precision_0_std', 
                                 'precision_1_mean', 'precision_1_std', 'recall_0_mean', 'recall_0_std', 
                                 'recall_1_mean','recall_1_std', 'F1_0_mean', 'F1_0_std', 
                                 'F1_1_mean', 'F1_1_std']                                 
          
        
                
        MLPmaxacc = MLPaverage_df[MLPaverage_df.acc_mean == MLPaverage_df.acc_mean.max()]
        MLPmaxacc_copy  = MLPmaxacc.copy()
        MLPmaxacc_copy['x_col'] = x       
        
    #SVM1 CLassifier
        SVM1 = SVC(kernel="linear", C=0.025)
        SVM1.fit(X_train, y_resampled)
        pred_SVM1 = SVM1.predict(test_X)
        SVM1Results_df = classificationMetrics(SVM1Results_df, test_y, pred_SVM1)  
    
        #Determine the mean accuracy of the different dendrogram settings
        acc_mean = SVM1Results_df.groupby('y')['acc'].mean()
        acc_std = SVM1Results_df.groupby('y')['acc'].std()
        precision_0_mean = SVM1Results_df.groupby('y')['precision 0'].mean()
        precision_0_std = SVM1Results_df.groupby('y')['precision 0'].std()        
        precision_1_mean = SVM1Results_df.groupby('y')['precision 1'].mean()
        precision_1_std = SVM1Results_df.groupby('y')['precision 1'].std()
        recall_0_mean = SVM1Results_df.groupby('y')['recall 0'].mean()
        recall_0_std = SVM1Results_df.groupby('y')['recall 0'].std()      
        recall_1_mean = SVM1Results_df.groupby('y')['recall 1'].mean()
        recall_1_std = SVM1Results_df.groupby('y')['recall 1'].std()
        F1_0_mean = SVM1Results_df.groupby('y')['F1 0'].mean()
        F1_0_std = SVM1Results_df.groupby('y')['F1 0'].std()
        F1_1_mean = SVM1Results_df.groupby('y')['F1 1'].mean()
        F1_1_std = SVM1Results_df.groupby('y')['F1 1'].std()
        
        SVM1average_df = pd.concat([acc_mean , acc_std, 
                                   precision_0_mean, precision_0_std, 
                                   precision_1_mean, precision_1_std, 
                                   recall_0_mean, recall_0_std, 
                                   recall_1_mean, recall_1_std,
                                   F1_0_mean, F1_0_std,
                                   F1_1_mean, F1_1_std], axis=1)

        SVM1average_df.columns = ['acc_mean' , 'acc_std', 'precision_0_mean', 'precision_0_std', 
                                 'precision_1_mean', 'precision_1_std', 'recall_0_mean', 'recall_0_std', 
                                 'recall_1_mean','recall_1_std', 'F1_0_mean', 'F1_0_std', 
                                 'F1_1_mean', 'F1_1_std']                                 
                              
        SVM1maxacc = SVM1average_df[SVM1average_df.acc_mean == SVM1average_df.acc_mean.max()]
        SVM1maxacc_copy  = SVM1maxacc.copy()
        SVM1maxacc_copy['x_col'] = x       
        
    #SVM2 CLassifier
        SVM2 = SVC(gamma=2, C=1)
        SVM2.fit(X_train, y_resampled)
        pred_SVM2 = SVM2.predict(test_X)
        SVM2Results_df = classificationMetrics(SVM2Results_df, test_y, pred_SVM2)
            #Determine the mean accuracy of the different dendrogram settings
        acc_mean = SVM2Results_df.groupby('y')['acc'].mean()
        acc_std = SVM2Results_df.groupby('y')['acc'].std()
        precision_0_mean = SVM2Results_df.groupby('y')['precision 0'].mean()
        precision_0_std = SVM2Results_df.groupby('y')['precision 0'].std()        
        precision_1_mean = SVM2Results_df.groupby('y')['precision 1'].mean()
        precision_1_std = SVM2Results_df.groupby('y')['precision 1'].std()
        recall_0_mean = SVM2Results_df.groupby('y')['recall 0'].mean()
        recall_0_std = SVM2Results_df.groupby('y')['recall 0'].std()      
        recall_1_mean = SVM2Results_df.groupby('y')['recall 1'].mean()
        recall_1_std = SVM2Results_df.groupby('y')['recall 1'].std()
        F1_0_mean = SVM2Results_df.groupby('y')['F1 0'].mean()
        F1_0_std = SVM2Results_df.groupby('y')['F1 0'].std()
        F1_1_mean = SVM2Results_df.groupby('y')['F1 1'].mean()
        F1_1_std = SVM2Results_df.groupby('y')['F1 1'].std()
        
        SVM2average_df = pd.concat([acc_mean , acc_std, 
                                   precision_0_mean, precision_0_std, 
                                   precision_1_mean, precision_1_std, 
                                   recall_0_mean, recall_0_std, 
                                   recall_1_mean, recall_1_std,
                                   F1_0_mean, F1_0_std,
                                   F1_1_mean, F1_1_std], axis=1)

        SVM2average_df.columns = ['acc_mean' , 'acc_std', 'precision_0_mean', 'precision_0_std', 
                                 'precision_1_mean', 'precision_1_std', 'recall_0_mean', 'recall_0_std', 
                                 'recall_1_mean','recall_1_std', 'F1_0_mean', 'F1_0_std', 
                                 'F1_1_mean', 'F1_1_std']
        
        SVM2maxacc = SVM2average_df[SVM2average_df.acc_mean == SVM2average_df.acc_mean.max()]
        SVM2maxacc_copy  = SVM2maxacc.copy()
        SVM2maxacc_copy['x_col'] = x
          
        
    #Gaussian Process CLassifier
        Gauss = GaussianProcessClassifier(1.0 * RBF(1.0))
        Gauss.fit(X_train, y_resampled)
        pred_Gauss = Gauss.predict(test_X)
        GaussResults_df = classificationMetrics(GaussResults_df, test_y, pred_Gauss)
    
        #Determine the mean accuracy of the different dendrogram settings
        acc_mean = GaussResults_df.groupby('y')['acc'].mean()
        acc_std = GaussResults_df.groupby('y')['acc'].std()
        precision_0_mean = GaussResults_df.groupby('y')['precision 0'].mean()
        precision_0_std = GaussResults_df.groupby('y')['precision 0'].std()        
        precision_1_mean = GaussResults_df.groupby('y')['precision 1'].mean()
        precision_1_std = GaussResults_df.groupby('y')['precision 1'].std()
        recall_0_mean = GaussResults_df.groupby('y')['recall 0'].mean()
        recall_0_std = GaussResults_df.groupby('y')['recall 0'].std()      
        recall_1_mean = GaussResults_df.groupby('y')['recall 1'].mean()
        recall_1_std = GaussResults_df.groupby('y')['recall 1'].std()
        F1_0_mean = GaussResults_df.groupby('y')['F1 0'].mean()
        F1_0_std = GaussResults_df.groupby('y')['F1 0'].std()
        F1_1_mean = GaussResults_df.groupby('y')['F1 1'].mean()
        F1_1_std = GaussResults_df.groupby('y')['F1 1'].std()
        
        Gaussaverage_df = pd.concat([acc_mean , acc_std, 
                                   precision_0_mean, precision_0_std, 
                                   precision_1_mean, precision_1_std, 
                                   recall_0_mean, recall_0_std, 
                                   recall_1_mean, recall_1_std,
                                   F1_0_mean, F1_0_std,
                                   F1_1_mean, F1_1_std], axis=1)

        Gaussaverage_df.columns = ['acc_mean' , 'acc_std', 'precision_0_mean', 'precision_0_std', 
                                 'precision_1_mean', 'precision_1_std', 'recall_0_mean', 'recall_0_std', 
                                 'recall_1_mean','recall_1_std', 'F1_0_mean', 'F1_0_std', 
                                 'F1_1_mean', 'F1_1_std']                                 

        Gaussmaxacc = Gaussaverage_df[Gaussaverage_df.acc_mean == Gaussaverage_df.acc_mean.max()]
        Gaussmaxacc_copy  = Gaussmaxacc.copy()
        Gaussmaxacc_copy['x_col'] = x
          
    #Decision Tree CLassifier
        DT = DecisionTreeClassifier(max_depth=9)
        DT.fit(X_train, y_resampled)
        pred_DT = DT.predict(test_X)
        DTResults_df = classificationMetrics(DTResults_df, test_y, pred_DT)
 
        #Determine the mean accuracy of the different dendrogram settings
        acc_mean = DTResults_df.groupby('y')['acc'].mean()
        acc_std = DTResults_df.groupby('y')['acc'].std()
        precision_0_mean = DTResults_df.groupby('y')['precision 0'].mean()
        precision_0_std = DTResults_df.groupby('y')['precision 0'].std()        
        precision_1_mean = DTResults_df.groupby('y')['precision 1'].mean()
        precision_1_std = DTResults_df.groupby('y')['precision 1'].std()
        recall_0_mean = DTResults_df.groupby('y')['recall 0'].mean()
        recall_0_std = DTResults_df.groupby('y')['recall 0'].std()      
        recall_1_mean = DTResults_df.groupby('y')['recall 1'].mean()
        recall_1_std = DTResults_df.groupby('y')['recall 1'].std()
        F1_0_mean = DTResults_df.groupby('y')['F1 0'].mean()
        F1_0_std = DTResults_df.groupby('y')['F1 0'].std()
        F1_1_mean = DTResults_df.groupby('y')['F1 1'].mean()
        F1_1_std = DTResults_df.groupby('y')['F1 1'].std()
        
        DTaverage_df = pd.concat([acc_mean , acc_std, 
                                   precision_0_mean, precision_0_std, 
                                   precision_1_mean, precision_1_std, 
                                   recall_0_mean, recall_0_std, 
                                   recall_1_mean, recall_1_std,
                                   F1_0_mean, F1_0_std,
                                   F1_1_mean, F1_1_std], axis=1)

        DTaverage_df.columns = ['acc_mean' , 'acc_std', 'precision_0_mean', 'precision_0_std', 
                                 'precision_1_mean', 'precision_1_std', 'recall_0_mean', 'recall_0_std', 
                                 'recall_1_mean','recall_1_std', 'F1_0_mean', 'F1_0_std', 
                                 'F1_1_mean', 'F1_1_std']                                 
          
        
                
        DTmaxacc = DTaverage_df[DTaverage_df.acc_mean == DTaverage_df.acc_mean.max()]
        DTmaxacc_copy  = DTmaxacc.copy()
        DTmaxacc_copy['x_col'] = x       
        
    #Ada Boost CLassifier
        ada = AdaBoostClassifier()
        ada.fit(X_train, y_resampled)
        pred_ada = ada.predict(test_X)
        adaResults_df = classificationMetrics(adaResults_df, test_y, pred_ada)
 
        #Determine the mean accuracy of the different dendrogram settings
        acc_mean = adaResults_df.groupby('y')['acc'].mean()
        acc_std = adaResults_df.groupby('y')['acc'].std()
        precision_0_mean = adaResults_df.groupby('y')['precision 0'].mean()
        precision_0_std = adaResults_df.groupby('y')['precision 0'].std()        
        precision_1_mean = adaResults_df.groupby('y')['precision 1'].mean()
        precision_1_std = adaResults_df.groupby('y')['precision 1'].std()
        recall_0_mean = adaResults_df.groupby('y')['recall 0'].mean()
        recall_0_std = adaResults_df.groupby('y')['recall 0'].std()      
        recall_1_mean = adaResults_df.groupby('y')['recall 1'].mean()
        recall_1_std = adaResults_df.groupby('y')['recall 1'].std()
        F1_0_mean = adaResults_df.groupby('y')['F1 0'].mean()
        F1_0_std = adaResults_df.groupby('y')['F1 0'].std()
        F1_1_mean = adaResults_df.groupby('y')['F1 1'].mean()
        F1_1_std = adaResults_df.groupby('y')['F1 1'].std()
        
        adaaverage_df = pd.concat([acc_mean , acc_std, 
                                   precision_0_mean, precision_0_std, 
                                   precision_1_mean, precision_1_std, 
                                   recall_0_mean, recall_0_std, 
                                   recall_1_mean, recall_1_std,
                                   F1_0_mean, F1_0_std,
                                   F1_1_mean, F1_1_std], axis=1)

        adaaverage_df.columns = ['acc_mean' , 'acc_std', 'precision_0_mean', 'precision_0_std', 
                                 'precision_1_mean', 'precision_1_std', 'recall_0_mean', 'recall_0_std', 
                                 'recall_1_mean','recall_1_std', 'F1_0_mean', 'F1_0_std', 
                                 'F1_1_mean', 'F1_1_std']                                 
        adamaxacc = adaaverage_df[adaaverage_df.acc_mean == adaaverage_df.acc_mean.max()]
        adamaxacc_copy  = adamaxacc.copy()
        adamaxacc_copy['x_col'] = x
        
    #Naive Bayes CLassifier
        NB = GaussianNB()
        NB.fit(X_train, y_resampled)
        pred_NB = NB.predict(test_X)
        NBResults_df = classificationMetrics(NBResults_df, test_y, pred_NB)
 
            #Determine the mean accuracy of the different dendrogram settings
        acc_mean = NBResults_df.groupby('y')['acc'].mean()
        acc_std = NBResults_df.groupby('y')['acc'].std()
        precision_0_mean = NBResults_df.groupby('y')['precision 0'].mean()
        precision_0_std = NBResults_df.groupby('y')['precision 0'].std()        
        precision_1_mean = NBResults_df.groupby('y')['precision 1'].mean()
        precision_1_std = NBResults_df.groupby('y')['precision 1'].std()
        recall_0_mean = NBResults_df.groupby('y')['recall 0'].mean()
        recall_0_std = NBResults_df.groupby('y')['recall 0'].std()      
        recall_1_mean = NBResults_df.groupby('y')['recall 1'].mean()
        recall_1_std = NBResults_df.groupby('y')['recall 1'].std()
        F1_0_mean = NBResults_df.groupby('y')['F1 0'].mean()
        F1_0_std = NBResults_df.groupby('y')['F1 0'].std()
        F1_1_mean = NBResults_df.groupby('y')['F1 1'].mean()
        F1_1_std = NBResults_df.groupby('y')['F1 1'].std()
        
        NBaverage_df = pd.concat([acc_mean , acc_std, 
                                   precision_0_mean, precision_0_std, 
                                   precision_1_mean, precision_1_std, 
                                   recall_0_mean, recall_0_std, 
                                   recall_1_mean, recall_1_std,
                                   F1_0_mean, F1_0_std,
                                   F1_1_mean, F1_1_std], axis=1)

        NBaverage_df.columns = ['acc_mean' , 'acc_std', 'precision_0_mean', 'precision_0_std', 
                                 'precision_1_mean', 'precision_1_std', 'recall_0_mean', 'recall_0_std', 
                                 'recall_1_mean','recall_1_std', 'F1_0_mean', 'F1_0_std', 
                                 'F1_1_mean', 'F1_1_std']                                 

        NBmaxacc = NBaverage_df[NBaverage_df.acc_mean == NBaverage_df.acc_mean.max()]
        NBmaxacc_copy  = NBmaxacc.copy()
        NBmaxacc_copy['x_col'] = x
        
        #Logistic CLassifier
        clf = LogisticRegression(solver = 'liblinear', penalty = "l1", C=0.01)
        clf.fit(X_train, y_resampled)
        pred_LR = clf.predict(test_X)
        LRResults_df = classificationMetrics(LRResults_df, test_y, pred_LR)
        
        
        #Determine the mean accuracy of the different dendrogram settings
        acc_mean = LRResults_df.groupby('y')['acc'].mean()
        acc_std = LRResults_df.groupby('y')['acc'].std()
        precision_0_mean = LRResults_df.groupby('y')['precision 0'].mean()
        precision_0_std = LRResults_df.groupby('y')['precision 0'].std()        
        precision_1_mean = LRResults_df.groupby('y')['precision 1'].mean()
        precision_1_std = LRResults_df.groupby('y')['precision 1'].std()
        recall_0_mean = LRResults_df.groupby('y')['recall 0'].mean()
        recall_0_std = LRResults_df.groupby('y')['recall 0'].std()      
        recall_1_mean = LRResults_df.groupby('y')['recall 1'].mean()
        recall_1_std = LRResults_df.groupby('y')['recall 1'].std()
        F1_0_mean = LRResults_df.groupby('y')['F1 0'].mean()
        F1_0_std = LRResults_df.groupby('y')['F1 0'].std()
        F1_1_mean = LRResults_df.groupby('y')['F1 1'].mean()
        F1_1_std = LRResults_df.groupby('y')['F1 1'].std()
        
        LRaverage_df = pd.concat([acc_mean , acc_std, 
                                   precision_0_mean, precision_0_std, 
                                   precision_1_mean, precision_1_std, 
                                   recall_0_mean, recall_0_std, 
                                   recall_1_mean, recall_1_std,
                                   F1_0_mean, F1_0_std,
                                   F1_1_mean, F1_1_std], axis=1)

        LRaverage_df.columns = ['acc_mean' , 'acc_std', 'precision_0_mean', 'precision_0_std', 
                                 'precision_1_mean', 'precision_1_std', 'recall_0_mean', 'recall_0_std', 
                                 'recall_1_mean','recall_1_std', 'F1_0_mean', 'F1_0_std', 
                                 'F1_1_mean', 'F1_1_std']                                 

        LRmaxacc = LRaverage_df[LRaverage_df.acc_mean == LRaverage_df.acc_mean.max()]
        LRmaxacc_copy  = LRmaxacc.copy()
        LRmaxacc_copy['x_col'] = x
        
             
    MLPmaxacc_comb = MLPmaxacc_comb.append(MLPmaxacc_copy)
    SVM1maxacc_comb = SVM1maxacc_comb.append(SVM1maxacc_copy)
    SVM2maxacc_comb = SVM2maxacc_comb.append(SVM2maxacc_copy)
    Gaussmaxacc_comb = Gaussmaxacc_comb.append(Gaussmaxacc_copy)
    DTmaxacc_comb = DTmaxacc_comb.append(DTmaxacc_copy)
    adamaxacc_comb = adamaxacc_comb.append(adamaxacc_copy)
    NBmaxacc_comb = NBmaxacc_comb.append(NBmaxacc_copy)
    LRmaxacc_comb = LRmaxacc_comb.append(LRmaxacc_copy)

#print(model_columns)
#model_columns.to_csv("model_columns.csv", index=False)

#Write the results onto a CSV file, currently commented out 
MLPmaxacc_comb.to_csv("10Runs_ClassificationScreening.csv",)
SVM1maxacc_comb.to_csv("10Runs_ClassificationScreening.csv", mode="a")
SVM2maxacc_comb.to_csv("10Runs_ClassificationScreening.csv", mode="a")
Gaussmaxacc_comb.to_csv("10Runs_ClassificationScreening.csv", mode="a")
DTmaxacc_comb.to_csv("10Runs_ClassificationScreening.csv", mode="a")
adamaxacc_comb.to_csv("10Runs_ClassificationScreening.csv", mode="a")
NBmaxacc_comb.to_csv("10Runs_ClassificationScreening.csv", mode="a")
LRmaxacc_comb.to_csv("10Runs_ClassificationScreening.csv", mode="a")


/home/sweav003/envs/CHB/lib/python3.7/site-packages/sklearn/svm/_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/home/sweav003/envs/CHB/lib/python3.7/site-packages/sklearn/svm/_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/home/sweav003/envs/CHB/lib/python3.7/site-packages/sklearn/svm/_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [14]:
MLPmaxacc_comb

,acc_mean,acc_std,precision_0_mean,precision_0_std,precision_1_mean,precision_1_std,recall_0_mean,recall_0_std,recall_1_mean,recall_1_std,F1_0_mean,F1_0_std,F1_1_mean,F1_1_std,x_col
y,,,,,,,,,,,,,,,
0.5,0.648804,0.077124,0.871996,0.025784,0.293636,0.030603,0.668047,0.135979,0.567500,0.185611,0.747624,0.079889,0.373966,0.055728,1
0.5,0.658128,0.090931,0.861822,0.024267,0.290538,0.036804,0.693617,0.154539,0.506250,0.195488,0.757179,0.093724,0.350033,0.057274,2
0.5,0.617426,0.109192,0.862008,0.035863,0.284104,0.033130,0.629078,0.182397,0.560000,0.200388,0.707799,0.138850,0.359897,0.051586,3
0.5,0.632027,0.100854,0.862359,0.033270,0.286652,0.030912,0.651048,0.169152,0.545000,0.191586,0.725395,0.126474,0.360202,0.050459,4
0.5,0.607326,0.110115,0.859852,0.031142,0.285659,0.029532,0.610103,0.185098,0.580000,0.194897,0.694094,0.140780,0.367921,0.048395,5
0.5,0.594812,0.121148,0.860090,0.037416,0.284511,0.033150,0.592569,0.205841,0.588333,0.214539,0.675316,0.162083,0.365931,0.049305,6
0.5,0.597717,0.118046,0.860263,0.036218,0.275115,0.040816,0.599838,0.200586,0.572857,0.218598,0.682169,0.155244,0.354288,0.060148,7
0.5,0.594588,0.122345,0.856529,0.036182,0.272632,0.039744,0.598478,0.207580,0.565000,0.224933,0.678275,0.161288,0.348979,0.061662,8
0.5,0.595496,0.122276,0.858545,0.035653,0.272699,0.041219,0.599212,0.205730,0.568056,0.221335,0.680085,0.158855,0.349493,0.058360,9


In [15]:
SVM1maxacc_comb

,acc_mean,acc_std,precision_0_mean,precision_0_std,precision_1_mean,precision_1_std,recall_0_mean,recall_0_std,recall_1_mean,recall_1_std,F1_0_mean,F1_0_std,F1_1_mean,F1_1_std,x_col
y,,,,,,,,,,,,,,,
0.40,0.588517,0.000000,0.867257,0.000000,0.260417,0.000000,0.579882,0.000000,0.625000,0.000000,0.695035,0.000000,0.367647,0.000000,1
0.50,0.588517,0.000000,0.860870,0.000000,0.255319,0.000000,0.585799,0.000000,0.600000,0.000000,0.697183,0.000000,0.358209,0.000000,1
0.35,0.626738,0.049032,0.876041,0.011467,0.285779,0.031474,0.627047,0.060532,0.625000,0.000000,0.729742,0.045172,0.391243,0.029676,2
0.30,0.585394,0.053759,0.852184,0.037547,0.263593,0.025848,0.581823,0.060737,0.600000,0.062284,0.690531,0.051703,0.365677,0.032702,3
0.40,0.605470,0.069770,0.853583,0.012512,0.268365,0.012721,0.612236,0.108360,0.568750,0.088388,0.707945,0.078421,0.361568,0.010948,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0.55,0.583365,0.077632,0.842815,0.013187,0.267288,0.017471,0.581815,0.124670,0.575000,0.112747,0.681405,0.088765,0.360415,0.025997,6
0.30,0.574659,0.070054,0.858616,0.027693,0.264058,0.021333,0.557979,0.114260,0.625000,0.121881,0.669358,0.090738,0.369231,0.037725,7
0.30,0.574992,0.065476,0.859707,0.026044,0.266207,0.020742,0.557225,0.106802,0.631250,0.115116,0.670014,0.084819,0.372709,0.036453,8


In [16]:
SVM2maxacc_comb

,acc_mean,acc_std,precision_0_mean,precision_0_std,precision_1_mean,precision_1_std,recall_0_mean,recall_0_std,recall_1_mean,recall_1_std,F1_0_mean,F1_0_std,F1_1_mean,F1_1_std,x_col
y,,,,,,,,,,,,,,,
0.25,0.808612,0.000000,0.808612,0.000000,0.000000,0.000000,1.0,0.0,0.000000,0.000000,0.894180,0.000000,0.000000,0.000000,1
0.30,0.808612,0.000000,0.808612,0.000000,0.000000,0.000000,1.0,0.0,0.000000,0.000000,0.894180,0.000000,0.000000,0.000000,1
0.35,0.808612,0.000000,0.808612,0.000000,0.000000,0.000000,1.0,0.0,0.000000,0.000000,0.894180,0.000000,0.000000,0.000000,1
0.40,0.808612,0.000000,0.808612,0.000000,0.000000,0.000000,1.0,0.0,0.000000,0.000000,0.894180,0.000000,0.000000,0.000000,1
0.45,0.808612,0.000000,0.808612,0.000000,0.000000,0.000000,1.0,0.0,0.000000,0.000000,0.894180,0.000000,0.000000,0.000000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0.55,0.795699,0.016657,0.795552,0.016485,0.166667,0.375823,1.0,0.0,0.004167,0.009396,0.886044,0.010239,0.008130,0.018333,6
0.55,0.800148,0.018913,0.800022,0.018814,0.142857,0.352454,1.0,0.0,0.003571,0.008811,0.888783,0.011631,0.006969,0.017193,7
0.55,0.799356,0.017801,0.799246,0.017704,0.125000,0.332805,1.0,0.0,0.003125,0.008320,0.888317,0.010941,0.006098,0.016234,8


In [17]:
Gaussmaxacc_comb

,acc_mean,acc_std,precision_0_mean,precision_0_std,precision_1_mean,precision_1_std,recall_0_mean,recall_0_std,recall_1_mean,recall_1_std,F1_0_mean,F1_0_std,F1_1_mean,F1_1_std,x_col
y,,,,,,,,,,,,,,,
0.25,0.808612,0.000000,0.808612,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.894180,0.000000,0.000000,0.000000,1
0.35,0.808612,0.000000,0.808612,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.894180,0.000000,0.000000,0.000000,1
0.40,0.808612,0.000000,0.808612,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.894180,0.000000,0.000000,0.000000,1
0.45,0.808612,0.000000,0.808612,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.894180,0.000000,0.000000,0.000000,1
0.35,0.812325,0.003809,0.811889,0.003362,0.500000,0.512989,1.000000,0.000000,0.012500,0.012825,0.896176,0.002048,0.024390,0.025024,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0.25,0.793907,0.018248,0.795163,0.016761,0.166667,0.375823,0.997717,0.005148,0.004167,0.009396,0.884912,0.011241,0.008130,0.018333,6
0.35,0.797990,0.019684,0.800022,0.018598,0.178571,0.348763,0.996317,0.007169,0.007143,0.011375,0.887329,0.012276,0.013462,0.021458,7
0.40,0.796876,0.018091,0.799133,0.017432,0.156250,0.331316,0.996046,0.006739,0.006250,0.010894,0.886682,0.011273,0.011779,0.020548,8


In [18]:
DTmaxacc_comb

,acc_mean,acc_std,precision_0_mean,precision_0_std,precision_1_mean,precision_1_std,recall_0_mean,recall_0_std,recall_1_mean,recall_1_std,F1_0_mean,F1_0_std,F1_1_mean,F1_1_std,x_col
y,,,,,,,,,,,,,,,
0.30,0.798086,0.010036,0.869055,0.006885,0.470307,0.028513,0.883432,0.006861,0.437500,0.031732,0.876174,0.006123,0.453202,0.029504,1
0.25,0.779215,0.021275,0.877475,0.014214,0.435919,0.033338,0.846637,0.045366,0.492500,0.085494,0.860791,0.018184,0.456657,0.022755,2
0.35,0.779296,0.019228,0.892163,0.013576,0.454996,0.053175,0.824888,0.023934,0.591667,0.083132,0.856926,0.012710,0.513084,0.061799,3
0.35,0.775823,0.017964,0.885232,0.017017,0.439448,0.053883,0.829338,0.022281,0.552500,0.099808,0.856048,0.011271,0.487999,0.069598,4
0.35,0.767249,0.023682,0.889836,0.017807,0.441123,0.048247,0.809059,0.045611,0.592000,0.119570,0.846469,0.021876,0.502554,0.068727,5
0.35,0.761961,0.024987,0.891018,0.016871,0.439974,0.044395,0.798808,0.047812,0.608750,0.116064,0.841286,0.023316,0.507726,0.064118,6
0.35,0.761272,0.023591,0.896732,0.021289,0.433120,0.044885,0.793668,0.046133,0.626071,0.116286,0.840895,0.021799,0.508054,0.059912,7
0.35,0.760443,0.022403,0.894646,0.020828,0.432680,0.042383,0.794459,0.043263,0.620313,0.110242,0.840544,0.020515,0.506233,0.056677,8
0.30,0.760245,0.027879,0.879256,0.016580,0.420276,0.061116,0.812624,0.048059,0.539444,0.116357,0.843594,0.023766,0.468636,0.071956,9


In [19]:
adamaxacc_comb

,acc_mean,acc_std,precision_0_mean,precision_0_std,precision_1_mean,precision_1_std,recall_0_mean,recall_0_std,recall_1_mean,recall_1_std,F1_0_mean,F1_0_std,F1_1_mean,F1_1_std,x_col
y,,,,,,,,,,,,,,,
0.45,0.803828,0.000000,0.905063,0.000000,0.490196,0.000000,0.846154,0.000000,0.625000,0.000000,0.874618,0.000000,0.549451,0.000000,1
0.45,0.783989,0.020354,0.917320,0.012576,0.459384,0.031613,0.806798,0.040378,0.687500,0.064124,0.857691,0.017366,0.547453,0.002050,2
0.35,0.763569,0.016220,0.897640,0.014634,0.433469,0.005547,0.796414,0.041348,0.625000,0.095141,0.843097,0.017794,0.509046,0.034326,3
0.35,0.759881,0.015411,0.903587,0.016373,0.427214,0.011969,0.785907,0.040137,0.650000,0.093026,0.839718,0.016449,0.512415,0.030184,4
0.35,0.746541,0.030255,0.904869,0.014835,0.423350,0.013227,0.762549,0.059237,0.675000,0.097153,0.826012,0.031339,0.516829,0.028366,5
0.35,0.740515,0.030739,0.906836,0.014228,0.421681,0.012628,0.750842,0.060093,0.691667,0.096184,0.819817,0.031793,0.520546,0.027176,6
0.35,0.734912,0.031606,0.908403,0.013713,0.408500,0.034554,0.743803,0.058219,0.692857,0.088990,0.816308,0.030648,0.510182,0.035850,7
0.35,0.738409,0.030971,0.909276,0.013025,0.414078,0.035544,0.747419,0.055254,0.696875,0.083853,0.819023,0.029540,0.516121,0.037048,8
0.35,0.743409,0.032460,0.909511,0.012290,0.417033,0.034527,0.754768,0.056097,0.691667,0.080378,0.823543,0.030657,0.517135,0.035024,9


In [20]:
LRmaxacc_comb

,acc_mean,acc_std,precision_0_mean,precision_0_std,precision_1_mean,precision_1_std,recall_0_mean,recall_0_std,recall_1_mean,recall_1_std,F1_0_mean,F1_0_std,F1_1_mean,F1_1_std,x_col
y,,,,,,,,,,,,,,,
0.25,0.622010,0.000000,0.851562,0.000000,0.259259,0.000000,0.644970,0.000000,0.525000,0.000000,0.734007,0.000000,0.347107,0.000000,1
0.30,0.622010,0.000000,0.862903,0.000000,0.270588,0.000000,0.633136,0.000000,0.575000,0.000000,0.730375,0.000000,0.368000,0.000000,1
0.35,0.622010,0.000000,0.862903,0.000000,0.270588,0.000000,0.633136,0.000000,0.575000,0.000000,0.730375,0.000000,0.368000,0.000000,1
0.40,0.622010,0.000000,0.862903,0.000000,0.270588,0.000000,0.633136,0.000000,0.575000,0.000000,0.730375,0.000000,0.368000,0.000000,1
0.45,0.622010,0.000000,0.862903,0.000000,0.270588,0.000000,0.633136,0.000000,0.575000,0.000000,0.730375,0.000000,0.368000,0.000000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0.25,0.538825,0.074093,0.840284,0.034094,0.250799,0.009940,0.515342,0.114664,0.620833,0.102321,0.631795,0.095135,0.355298,0.018989,6
0.25,0.538535,0.068517,0.844303,0.033050,0.245496,0.015990,0.515768,0.106035,0.621429,0.094628,0.634279,0.088185,0.350037,0.021835,7
0.25,0.528563,0.069320,0.843217,0.031022,0.243734,0.015663,0.500810,0.106800,0.631250,0.092221,0.622175,0.088491,0.349761,0.020420,8


In [21]:
NBmaxacc_comb

,acc_mean,acc_std,precision_0_mean,precision_0_std,precision_1_mean,precision_1_std,recall_0_mean,recall_0_std,recall_1_mean,recall_1_std,F1_0_mean,F1_0_std,F1_1_mean,F1_1_std,x_col
y,,,,,,,,,,,,,,,
0.35,0.732057,0.000000,0.900709,0.000000,0.382353,0.000000,0.751479,0.000000,0.650000,0.000000,0.819355,0.000000,0.481481,0.000000,1
0.50,0.689108,0.039156,0.882789,0.011878,0.330317,0.043929,0.710042,0.042514,0.600000,0.025649,0.786650,0.030852,0.425312,0.043069,2
0.50,0.651162,0.063116,0.889731,0.013861,0.323771,0.036783,0.644594,0.100232,0.666667,0.098114,0.742774,0.067872,0.432725,0.036455,3
0.50,0.655433,0.054940,0.891708,0.012446,0.323186,0.031735,0.650112,0.086972,0.668750,0.084685,0.748355,0.059340,0.433415,0.031459,4
0.35,0.624174,0.084543,0.902156,0.010496,0.323833,0.035563,0.591832,0.132766,0.735000,0.095298,0.705652,0.098172,0.445599,0.026780,5
0.35,0.603934,0.089550,0.897035,0.014995,0.315584,0.037368,0.564289,0.136003,0.741667,0.088138,0.683505,0.102461,0.438871,0.028737,6
0.35,0.597435,0.084345,0.896574,0.013913,0.304399,0.044221,0.559219,0.126382,0.735714,0.082814,0.680786,0.094983,0.426808,0.039898,7
0.35,0.606519,0.082452,0.894671,0.013954,0.309112,0.043191,0.573732,0.124273,0.725000,0.082485,0.691276,0.093058,0.429492,0.037966,8
0.25,0.608450,0.059248,0.897064,0.018100,0.303547,0.033237,0.576525,0.091744,0.727778,0.085002,0.697478,0.069185,0.426425,0.037357,9
